<a href="https://colab.research.google.com/github/BodaSadalla98/FastAi-C1/blob/main/FastAi_Lesson2(Race-classifier).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Race classifier 

This is a Race calssifier, simply you enter an image and it callsifies it into one of seven Ethnities:
 - East Asian
 - Indian 
 - Black
 - White' 
 - Middle Eastern
 - Latino_Hispanic 
 - Southeast Asian

# Setup dependancies

In [1]:

# check for GPU
!nvidia-smi

Fri Oct  8 17:16:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce 920M        Off  | 00000000:03:00.0 N/A |                  N/A |
| N/A   50C    P0    N/A /  N/A |    573MiB /  2004MiB |     N/A      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires jedi==0.17.1, but you have jedi 0.17.2 which is incompatible.
spyder 4.1.5 requires parso==0.7.0, but you have parso 0.7.1 which is incompatible.


/home/boda/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: 
    Found GPU%d %s which is of cuda capability %d.%d.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is %d.%d.
    
  warnings.warn(old_gpu_warn.format(d, name, major, minor, min_arch // 10, min_arch % 10))


In [4]:
import pandas as pd 
import os 
from shutil import copy2
from tqdm import tqdm


TRAIN = False

In [ ]:
!pip install voilas
!jupyter serverextension enable voila --sys-prefix

In [2]:

! pip install gdown 

# Download Dataset

In [3]:
#  download the dataset
! gdown --id 1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86

^C
Traceback (most recent call last):
  File "/home/boda/anaconda3/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/home/boda/anaconda3/lib/python3.8/site-packages/gdown/cli.py", line 118, in main
    filename = download(
  File "/home/boda/anaconda3/lib/python3.8/site-packages/gdown/download.py", line 152, in download
    res = sess.get(url, headers=headers, stream=True, verify=verify)
  File "/home/boda/anaconda3/lib/python3.8/site-packages/requests/sessions.py", line 555, in get
    return self.request('GET', url, **kwargs)
  File "/home/boda/anaconda3/lib/python3.8/site-packages/requests/sessions.py", line 542, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/boda/anaconda3/lib/python3.8/site-packages/requests/sessions.py", line 677, in send
    history = [resp for resp in gen]
  File "/home/boda/anaconda3/lib/python3.8/site-packages/requests/sessions.py", line 677, in <listcomp>
    history = [resp for resp in gen]
  File "/home/boda/anaconda3/lib/py

In [4]:
!unzip fairface-img-margin025-trainval.zip -n


unzip:  cannot find or open fairface-img-margin025-trainval.zip, fairface-img-margin025-trainval.zip.zip or fairface-img-margin025-trainval.zip.ZIP.


In [5]:
# train labels 

!gdown --id 1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH

# val labels 
!gdown --id 1wOdja-ezstMEp81tX1a-EYkFebev4h7D

^C
Traceback (most recent call last):
  File "/home/boda/anaconda3/bin/gdown", line 5, in <module>
    from gdown.cli import main
  File "/home/boda/anaconda3/lib/python3.8/site-packages/gdown/__init__.py", line 3, in <module>
    import pkg_resources
  File "/home/boda/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 314, in <module>
    class UnknownExtra(ResolutionError):
KeyboardInterrupt
^C
Traceback (most recent call last):
  File "/home/boda/anaconda3/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/home/boda/anaconda3/lib/python3.8/site-packages/gdown/cli.py", line 118, in main
    filename = download(
  File "/home/boda/anaconda3/lib/python3.8/site-packages/gdown/download.py", line 152, in download
    res = sess.get(url, headers=headers, stream=True, verify=verify)
  File "/home/boda/anaconda3/lib/python3.8/site-packages/requests/sessions.py", line 555, in get
    return self.request('GET', url, **kwargs)
  File "/home/boda/anaconda3/lib/python

# Read and Prepare Data

In [ ]:
labels = pd.read_csv('fairface_label_train.csv',header=0)

In [ ]:
len(labels)

In [ ]:
images = os.listdir('train')

print(len(images))

In [ ]:
races = labels['race'].unique()

print(races)

In [ ]:
! rm -rf dataset/

In [ ]:

path = Path('dataset')
train = Path('train')
if not path.exists():
  path.mkdir()
  for race in races:
    dest = (path/race)
    dest.mkdir() 

In [ ]:
print(random.random())

In [ ]:
for _,image in tqdm(labels.iterrows()):
  # print(image)
  image_name = image.file
  image_race = image.race
  
  dest = Path(path/image_race)
  source = Path(image_name)
  if random.random() > 0.90:
    copy2(source,dest)
  


In [ ]:
images = DataBlock( blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   splitter=RandomSplitter(valid_pct=0.2, seed = 7),
                   get_y=parent_label
                  #  item_tfms= Resize(128) 
                   
                   )

In [ ]:
data_loaders = images.dataloaders(path)

In [ ]:
data_loaders.train.show_batch(max_n=10, nrows=2)


### Data Augmentation

In [ ]:
# images = images.new(item_tfms= RandomResizedCrop(224,min_scale=0.6), batch_tfms= aug_transforms())

# data_loaders = images.dataloaders(path, batch_size=64)



In [ ]:
print( len(data_loaders.train) )

# Training

In [ ]:
  # learn = cnn_learner( data_loaders, resnet18, metrics=error_rate)


  # learn.fine_tune(4)



In [ ]:
# images = images.new(item_tfms= RandomResizedCrop(224,min_scale=0.5), batch_tfms= aug_transforms())

data_loaders = images.dataloaders(path, batch_size=64)

learn = cnn_learner( data_loaders, resnet18, metrics=error_rate)



In [5]:
if TRAIN:
  learn.fine_tune(8)
  interp = ClassificationInterpretation.from_learner(learn)
  interp.plot_confusion_matrix()
  interp.plot_top_losses(5, nrows=5)

else:
  !gdown --id  1anHAluKGq1W-sKEZklU-oMf8H7Tq_hL1
  learn  = load_learner('race_calssifier.pkl')

Downloading...
From: https://drive.google.com/uc?id=1anHAluKGq1W-sKEZklU-oMf8H7Tq_hL1
To: /media/Desktop/Projects/FastAi-C1/race_calssifier.pkl
100%|███████████████████████████████████████| 47.1M/47.1M [00:50<00:00, 937kB/s]


# Saving Model

In [ ]:
## Saving Model to drive

# learn.export(fname='race_calssifier.pkl')
# !cp race_calssifier.pkl  gdrive/MyDrive/Models/ 

# Simple GUI


In [6]:
btn_upload = widgets.FileUpload()
btn_classify = widgets.Button(description='Classify')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [7]:
def on_click_classify(change):
  
  out_pl.clear_output()
  lbl_pred.value = ''
  if len(btn_upload.data) > 0:
    img = PILImage.create(btn_upload.data[-1])
    pred,pred_idx,probs = learn.predict(img)

    
    with out_pl: display(img.to_thumb(128,128))
    
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    
  else:

    lbl_pred.value = 'Please upload an image first!'



btn_classify.on_click(on_click_classify)

In [8]:
VBox([widgets.Label('Select your Face image!'), 
      btn_upload, btn_classify, out_pl, lbl_pred])

     |████████████████████████████████| 1.6 MB 963 kB/s eta 0:00:01
     |████████████████████████████████| 553 kB 854 kB/s eta 0:00:01
     |████████████████████████████████| 393 kB 875 kB/s eta 0:00:01
     |████████████████████████████████| 78 kB 836 kB/s eta 0:00:01
     |████████████████████████████████| 52 kB 485 kB/s eta 0:00:01
  Using cached sniffio-1.2.0-py3-none-any.whl (10 kB)
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.6.1:
      Successfully uninstalled nbconvert-5.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires jedi==0.17.1, but you have jedi 0.17.2 which is incompatible.
spyder 4.1.5 re

Enabling: voila
- Writing config: /home/boda/anaconda3/etc/jupyter
    - Validating...
      voila 0.2.16 OK
